In [1]:
!pip install tenseal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 12.8 MB/s eta 0:00:00


In [2]:
import tenseal as ts

context=ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
context.generate_galois_keys()
context.global_scale=2**40

In [3]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
# X, y = make_classification(n_samples=1000, n_features=10, n_classes=2, random_state=42)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = pd.read_csv('X_train1.csv')
X_train=X_train.to_numpy()
y_train = pd.read_csv('y_train1.csv')
y_train=y_train.to_numpy().reshape(-1)
X_test = pd.read_csv('X_test1.csv')
y_test = pd.read_csv('y_test1.csv')
y_test=y_test.to_numpy().reshape(-1)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


In [5]:
len(X_train)

400

In [6]:
len(X_test)

100

In [7]:
enc_X_train=[]
for i in X_train:
  p=ts.ckks_vector(context,i)
  enc_X_train.append(p)

In [8]:
enc_y_train=[]
for i in y_train:
  p=ts.ckks_vector(context,[i])
  enc_y_train.append(p)

In [9]:
def sigmoid(z):
  z=z.decrypt()
  z=round(z[0],7)
  x=1/(1+np.exp(-z))
  x=round(x,7)
  x=ts.ckks_vector(context,[x])
  return x

In [10]:
def relinearlizing_vector(vec):
  vec_2=[]
  vec=vec.decrypt()
  for i in vec:
    p=round(i,7)
    vec_2.append(p)
  vec_2=ts.ckks_vector(context,vec_2)
  return vec_2

In [11]:
def relinearlizing_scalar(sca):
  sca=sca.decrypt()
  sca=round(sca[0],7)
  sca=ts.ckks_vector(context,[sca])
  return sca

In [12]:
n_samples,n_attributes=X_train.shape
n_samples,n_attributes

(400, 15)

In [13]:
weights=np.zeros(n_attributes)
bias=0
lr=0.01
weights,bias

(array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), 0)

In [14]:
weights=ts.ckks_vector(context,weights)
bias=ts.ckks_vector(context,[bias])

In [15]:
ar=np.arange(n_samples)
for epoch in range(3):
  np.random.shuffle(ar)
  for i in ar:
    xi=enc_X_train[i]
    yi=enc_y_train[i]
    lin_op=xi.dot(weights)
    lin_op=lin_op+bias
    y_pred=sigmoid(lin_op)
    db=y_pred-yi
    dw=db*xi
    dw=relinearlizing_vector(dw)
    db=relinearlizing_scalar(db)
    weights=relinearlizing_vector(weights)
    bias=relinearlizing_scalar(bias)
    weights=weights-(lr*dw)
    bias=bias-(lr*db)
    print(f'epoch={epoch}, {i} loop is executed')

epoch=0, 118 loop is executed
epoch=0, 392 loop is executed
epoch=0, 27 loop is executed
epoch=0, 128 loop is executed
epoch=0, 166 loop is executed
epoch=0, 55 loop is executed
epoch=0, 44 loop is executed
epoch=0, 42 loop is executed
epoch=0, 140 loop is executed
epoch=0, 201 loop is executed
epoch=0, 390 loop is executed
epoch=0, 298 loop is executed
epoch=0, 183 loop is executed
epoch=0, 359 loop is executed
epoch=0, 282 loop is executed
epoch=0, 46 loop is executed
epoch=0, 194 loop is executed
epoch=0, 176 loop is executed
epoch=0, 225 loop is executed
epoch=0, 304 loop is executed
epoch=0, 310 loop is executed
epoch=0, 196 loop is executed
epoch=0, 246 loop is executed
epoch=0, 301 loop is executed
epoch=0, 286 loop is executed
epoch=0, 126 loop is executed
epoch=0, 120 loop is executed
epoch=0, 264 loop is executed
epoch=0, 104 loop is executed
epoch=0, 69 loop is executed
epoch=0, 19 loop is executed
epoch=0, 360 loop is executed
epoch=0, 168 loop is executed
epoch=0, 355 loop

In [16]:
weights.decrypt()

[0.17232462055055342,
 0.24433339204179882,
 -0.07555256720194321,
 0.08469219676262897,
 0.055565854433525505,
 -0.08774983729807442,
 0.12098120289592884,
 0.1116108664664221,
 -0.06575231351376933,
 -0.10034146290493312,
 0.30601799656381534,
 0.11167990261670906,
 -0.08760412238059563,
 0.018580631067389643,
 0.16429609929764155]

In [17]:
bias.decrypt()

[-1.5537599131763233]

In [18]:
from sklearn.linear_model import LogisticRegression

In [19]:
model = LogisticRegression(max_iter=3, solver='saga',C=100)
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogisticRegression(C=100, max_iter=3, solver='saga')

In [20]:
weights_sklearn = model.coef_[0]
bias_sklearn = model.intercept_[0]

In [21]:
print("Weights from scikit-learn:", weights_sklearn)
print("Bias from scikit-learn:", bias_sklearn)

Weights from scikit-learn: [ 0.31126946  0.32071724 -0.05987472  0.08911101  0.0912813  -0.0681033
  0.20616103  0.09733893 -0.02465199 -0.07684827  0.42864729  0.10956113
 -0.17395778 -0.03318984  0.13862155]
Bias from scikit-learn: -1.8228600610322299


In [22]:
print("\nComparison of Weights and Bias:")
print("Weights difference:", np.abs(weights.decrypt() - weights_sklearn))
print("Bias difference:", np.abs(bias.decrypt() - bias_sklearn))


Comparison of Weights and Bias:
Weights difference: [0.13894484 0.07638385 0.01567785 0.00441882 0.03571544 0.01964654
 0.08517982 0.01427194 0.04110033 0.02349319 0.12262929 0.00211877
 0.08635366 0.05177047 0.02567455]
Bias difference: [0.26910015]


In [23]:
enc_X_test=[]
for i in X_test:
  p=ts.ckks_vector(context,i)
  enc_X_test.append(p)

In [24]:
y_pred_enc=[]
for i in enc_X_test:
  lin_op=i.dot(weights)
  lin_op=lin_op+bias
  y_pred=sigmoid(lin_op)
  y_pred_enc.append(y_pred)

In [25]:
y_pred_unenc=[]
for i in y_pred_enc:
  p=i.decrypt()[0]
  if p>=0.5:
    y_pred_unenc.append(1)
  else:
    y_pred_unenc.append(0)

In [26]:
for i in y_pred_enc:
  print(i.decrypt())

[0.24353050035328538]
[0.13022060196333163]
[0.1340450984322158]
[0.20718259948064363]
[0.2584393035196953]
[0.119399201135988]
[0.09508850345632795]
[0.10021160223198827]
[0.18759660102483225]
[0.1700411989525196]
[0.1791250001940338]
[0.38945050061549946]
[0.11119149972302374]
[0.22307860100288968]
[0.0921772027260835]
[0.21557200316681444]
[0.12571920298425807]
[0.14344850124132094]
[0.28648960132204165]
[0.12962089779167602]
[0.09255569996078633]
[0.20112270037379557]
[0.149039204774665]
[0.12121890026271924]
[0.3305101025264988]
[0.12281809683614107]
[0.1278208020969675]
[0.12643299783238252]
[0.2652602992262405]
[0.10603099625490454]
[0.3762990990623989]
[0.27019159740030657]
[0.1257429006099514]
[0.19651069599629511]
[0.21702469945079783]
[0.15064360009823805]
[0.28701600119516163]
[0.12869480655435112]
[0.17242230134027614]
[0.17517199978389034]
[0.18634810267958504]
[0.38648390574892305]
[0.11903210179371915]
[0.25308060045219016]
[0.1322832977314975]
[0.27301670368907577]
[0.

In [27]:
y_pred=model.predict(X_test)

In [28]:
len(y_pred_unenc)

100

In [29]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.83


In [30]:
accuracy = accuracy_score(y_test, y_pred_unenc)
print(f'Accuracy: {accuracy}')

Accuracy: 0.83
